In [1]:
#  This tutorial show that only pandas is used
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from waymo_open_dataset.utils import range_image_utils
import tensorflow as tf



ModuleNotFoundError: No module named 'waymo_open_dataset'

In [ ]:
# Path to the directory with all components
dataset_dir = '../WoD/validation'

context_name = '17065833287841703_2980_000_3000_000'

def read(tag: str) -> pd:
  """Creates a Dask DataFrame for the component specified by its tag."""
  paths = f'{dataset_dir}/{tag}/{context_name}.parquet'
  return pd.read_parquet(paths)

In [ ]:
WAYMO_TO_SEMKITTI_20 = {
    0: 0,   # UNKNOWN -> unlabeled
    1: 1,   # CAR -> car
    2: 4,   # TRUCK -> truck
    3: 4,   # BUS -> truck
    4: 5,   # OTHER_VEHICLE -> other-vehicle
    5: 8,   # MOTORCYCLIST -> motorcyclist
    6: 7,   # BICYCLIST -> bicyclist
    7: 6,   # PEDESTRIAN -> person
    8: 19,  # SIGN -> traffic-sign
    9: 19,  # TRAFFIC_LIGHT -> traffic-sign
    10: 18, # POLE -> pole
    11: 18, # CONSTRUCTION_CONE -> pole (approx)
    12: 2,  # BICYCLE -> bicycle
    13: 3,  # MOTORCYCLE -> motorcycle
    14: 13, # BUILDING -> building
    15: 15, # VEGETATION -> vegetation
    16: 16, # TREE_TRUNK -> trunk
    17: 12, # CURB -> other-ground (alt: road=9)
    18: 9,  # ROAD -> road
    19: 9,  # LANE_MARKER -> road
    20: 12, # OTHER_GROUND -> other-ground
    21: 11, # WALKABLE -> sidewalk
    22: 11, # SIDEWALK -> sidewalk
}

In [ ]:

def _sample_rgb_nearest(img, rows, cols):
    h, w = img.shape[:2]
    rows = np.clip(rows, 0, h - 1)
    cols = np.clip(cols, 0, w - 1)
    return img[rows, cols, :]

def colorize_range_image_v2(
    proj,                 # [H, W, 6] = [cam1,row1,col1, cam2,row2,col2]
    camera_images,        # dict: {camera_id:int -> np.uint8 [Hc,Wc,3]}
    default_color=(128,128,128)
):
    """
    Returns:
      colors_hw3: uint8 [H, W, 3]
      used_mask:  bool  [H, W]  (True where a camera projection was used)
    """
    H, W, C = proj.shape
    assert C == 6, "Expected v2 projection with two triplets: [cam1,row1,col1, cam2,row2,col2]"
    colors = np.empty((H, W, 3), dtype=np.uint8)
    colors[:] = np.array(default_color, dtype=np.uint8)
    used = np.zeros((H, W), dtype=bool)

    cam1 = proj[..., 0]; r1 = proj[..., 1]; c1 = proj[..., 2]
    cam2 = proj[..., 3]; r2 = proj[..., 4]; c2 = proj[..., 5]
    # SWAP row and col here!
    r1, c1 = c1, r1
    r2, c2 = c2, r2
    # Pass 1: try primary projection per camera
    for cid, img in camera_images.items():
        m1 = (cam1 == cid)
        if not np.any(m1):
            continue
        rr = r1[m1].astype(np.int32)
        cc = c1[m1].astype(np.int32)
        rgb = _sample_rgb_nearest(img, rr, cc)
        ii, jj = np.nonzero(m1)
        colors[ii, jj, :] = rgb
        used[ii, jj] = True

    # Pass 2: fallback to secondary where primary failed
    need = ~used
    for cid, img in camera_images.items():
        m2 = (cam2 == cid) & need
        if not np.any(m2):
            continue
        rr = r2[m2].astype(np.int32)
        cc = c2[m2].astype(np.int32)
        rgb = _sample_rgb_nearest(img, rr, cc)
        ii, jj = np.nonzero(m2)
        colors[ii, jj, :] = rgb
        used[ii, jj] = True

    return colors, used

def flatten_colors_for_points(used_mask, colors_hw3):
    """
    Align colors with flattened valid LiDAR pixels (same mask you'd use for ranges>0, etc.)
    Returns:
      colors_n3: uint8 [N, 3]
      idx_n2:    int   [N, 2]  (row, col) indices in the range image
    """
    ii, jj = np.nonzero(used_mask)
    return colors_hw3[ii, jj, :], np.stack([ii, jj], axis=1)

def decode_jpeg_image(img_proto):
    """Decode a Waymo camera image proto into uint8 HxWx3 (RGB)."""
    img = tf.image.decode_jpeg(img_proto, channels=3)
    return img.numpy()

In [ ]:
lidar_segmentation = read('lidar_segmentation')  # about 30 frames with label


In [ ]:
lidar_segmentation_first_frame = lidar_segmentation.iloc[0]
#  get the key.frame_timestamp_micros of this first frame
timestamp = lidar_segmentation_first_frame['key.frame_timestamp_micros']
# from now on, raw data, camera and label will be filtered by this timestamp

In [ ]:
# label
label_channel = lidar_segmentation_first_frame['[LiDARSegmentationLabelComponent].range_image_return1.values']
label_channel = label_channel.reshape(64, 2650, 2) # instance ID and semantic ID
lb1 = label_channel[:,:,1] # semantic ID: from 0 to 22
# Convert orig_sem_label to sem_label using the map
sem_label = np.vectorize(WAYMO_TO_SEMKITTI_20.get)(lb1)

In [ ]:
lidar_df = read('lidar')
# filter lidar_df that only has key.laser_name == 1 (first lidar) and key.frame_timestamp_micros == timestamp
lidar_df = lidar_df[lidar_df['key.laser_name'] == 1]
lidar_df = lidar_df[lidar_df['key.frame_timestamp_micros'] == timestamp]
# raw lidar data
lidar_raw = lidar_df.iloc[0]['[LiDARComponent].range_image_return1.values']
lidar_raw = lidar_raw.reshape(64, 2650, 4)    # range, intensity, elongation, is_in_no_label_zone
# remove elongation and is_in_no_label_zone
lidar_range = lidar_raw[:,:,0]
point_range = lidar_raw[:,:,0] 
point_intensity = lidar_raw[:,:,1]



In [ ]:
calib = read('lidar_calibration')
calib = calib[calib['key.laser_name'] == 1]
extrinsic = calib.iloc[0]['[LiDARCalibrationComponent].extrinsic.transform'].reshape(4, 4)
beam_inclination = calib.iloc[0]['[LiDARCalibrationComponent].beam_inclination.values']


In [ ]:
# convert lidar_range, extrinsic, and beam_inclination to tensors with [B, W, H] with B=1
lidar_range = tf.convert_to_tensor(lidar_range, dtype=tf.float32)
lidar_range = tf.expand_dims(lidar_range, axis=0)
extrinsic = tf.convert_to_tensor(extrinsic, dtype=tf.float32)
extrinsic = tf.expand_dims(extrinsic, axis=0)
beam_inclination = tf.convert_to_tensor(beam_inclination, dtype=tf.float32)
beam_inclination = tf.expand_dims(beam_inclination, axis=0)



In [ ]:
# Convert to 3D points (vehicle frame)
points = range_image_utils.extract_point_cloud_from_range_image(
    lidar_range,
    extrinsic,
    beam_inclination
)
#  make point to by numarray

points_np = points.numpy()
# squeeze to remove dimensions of size 1
points_np = points_np.squeeze()
# sanity check points_np: if point_range = -1, then points_np = -1
points_np[point_range == -1] = 0

In [ ]:
#  put the range image so that it has size {H, W} with five channel: range, x, y, z, intensity in such order
# lidar_raw[:,:,0] then points_np then lidar_raw[:,:,1]
points_final = np.zeros((64, 2650, 5))
points_final[:,:,0] = point_range
points_final[:,:,1:4] = points_np
points_final[:,:,4] = point_intensity
# now we have data that similar to that of the paper. Just need to add R,G, B (which is huge thing to do)



### Colorized point cloud

In [ ]:

cam_img_df = read('camera_image')
lidar_camera_projection_df = read('lidar_camera_projection')

In [ ]:
# filter using timestamp and key.laser_name
lidar_camera_projection_df = lidar_camera_projection_df[lidar_camera_projection_df['key.frame_timestamp_micros'] == timestamp]
lidar_camera_projection_df = lidar_camera_projection_df[lidar_camera_projection_df['key.laser_name'] == 1]
cam_img_df = cam_img_df[cam_img_df['key.frame_timestamp_micros'] == timestamp]


In [ ]:
# go through each row in cam_img_df, decode the image with [CameraImageComponent].image and key.camera_name (not that camera ID = key.camera_name - 1)
cam_images = {}
for index, row in cam_img_df.iterrows():
    cam_images[row['key.camera_name']] = decode_jpeg_image(row['[CameraImageComponent].image'])


In [ ]:
# get the [LiDARCameraProjectionComponent].range_image_return1.values
projection_df = lidar_camera_projection_df.iloc[0]['[LiDARCameraProjectionComponent].range_image_return1.values'].reshape(64, 2650,6)

In [ ]:

rgb_grid, used_mask = colorize_range_image_v2(projection_df, cam_images, default_color=(0,0,0))
pc_colors_n3, idx_n2 = flatten_colors_for_points(used_mask, rgb_grid)


In [ ]:
# combine the final points with range, x, y, z, intensity, flag, R, G, B and label
# flag is valid when and only when range > 0 and used_mask is one
points_final_output = np.zeros((64, 2650, 10))
points_final_output[:,:,0] = points_final[:,:,0]  # range
points_final_output[:,:,1] = points_final[:,:,1]  # x
points_final_output[:,:,2] = points_final[:,:,2]  # y
points_final_output[:,:,3] = points_final[:,:,3]  # z
points_final_output[:,:,4] = points_final[:,:,4]  # intensity
points_final_output[:,:,5] = (points_final[:,:,0] > 0) & (used_mask)  # flag
points_final_output[:,:,6] = rgb_grid[:,:, 0]  # R
points_final_output[:,:,7] = rgb_grid[:,:, 1]  # G
points_final_output[:,:,8] = rgb_grid[:,:, 2]  # B
points_final_output[:,:,9] = sem_label  # label (not used)

# sanity put: any -1 element is set to 0
points_final_output[points_final_output == -1] = 0
points_final_output.astype(np.float32).tofile("output_file.bin")